***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-cf9c  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  3 12:10 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '208593988'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "multistre" in b.name:
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_pairs = parquetFile.select("id", "anchor_text").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes:
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab.

In [ ]:
import numpy as np
from numpy.linalg import norm
from collections import Counter
import pyspark
from pyspark.sql import Row

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  word_count={}
  for token in tokens:
    if token not in word_count:
      word_count[token] = 1
    else:
      word_count[token] += 1

  tokens_list = []
  for k in word_count.keys():
    temp = (k, (id, word_count[k]))
    tokens_list.append(temp)
  return tokens_list

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  sorted_pl = sorted(unsorted_pl, key = lambda x: x[0])
  return sorted_pl

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  counts = postings.map(lambda x: (x[0], len(x[1])))
  counts.groupByKey()
  return counts

def partition_postings_and_write(postings, index_bins, folder_name):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''

  pl_to_block = postings.map(pl_bucket_mapper)
  pl_to_block = pl_to_block.reduceByKey(lambda a,b: [*a, *b])
  pl = pl_to_block.map(my_maper)
  my_index = InvertedIndex()
  posting_locs = pl.map(lambda b_w_pl: my_index.write_a_posting_list(b_w_pl, index_bins, folder_name))
  return posting_locs


def pl_bucket_mapper(item):
  return (token2bucket_id(item[0]), (item[0], item[1]))

def my_maper(item):
  item1 = item[1]
  n = len(item1) - 1
  lst = [(item1[i], item1[i+1]) for i in range(0, n, 2)]

  return (item[0], lst)

def doc_len(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  return (id, len(tokens))


def doc_norm(text, id, d):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  tokens_counter = Counter(tokens)
  vec = [item[1]/d[id] for item in tokens_counter.items() if d[id] != 0]


  return (id, norm(vec))

def calculate_tf(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  counts = postings.map(lambda x: (x[0], map_tf(x[1])))
  counts.groupByKey()
  return counts

def map_tf(pr):
    tf = 0
    for p in pr:
        tf += p[1]
    return tf

def map2(row):
    txt = ""
    anchors = row[1]
    id = row[0]

    for anchor in anchors:
        txt += anchor[1]
        txt += " "
    return Row(txt[:-1],id)

In [ ]:
# text index
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
docs_len = doc_text_pairs.map(lambda x: doc_len(x[0], x[1]))
docs_dict = docs_len.collectAsMap()
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
w2tf = calculate_tf(postings_filtered)
w2tf_dict = w2tf.collectAsMap()
docs_norm = doc_text_pairs.map(lambda x: doc_norm(x[0], x[1], docs_dict))
norm_dict = docs_norm.collectAsMap()


# partition posting lists and write out
folder_name = "postingText"
_ = partition_postings_and_write(postings_filtered, bucket_name, folder_name).collect()
super_posting_locs = defaultdict(list)

for blob in client.list_blobs(bucket_name, prefix='postingText'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

24/03/03 19:01:40 WARN BlockManagerMasterEndpoint: No more replicas available for broadcast_9_python !
24/03/03 19:01:40 WARN YarnAllocator: Container from a bad node: container_1709467795120_0004_01_000005 on host: cluster-cf9c-w-2.c.ir2024-416011.internal. Exit status: 137. Diagnostics: [2024-03-03 19:01:40.523]Container killed on request. Exit code is 137
[2024-03-03 19:01:40.526]Container exited with a non-zero exit code 137. 
[2024-03-03 19:01:40.527]Killed by external signal
.
24/03/03 19:01:40 ERROR YarnScheduler: Lost executor 5 on cluster-cf9c-w-2.c.ir2024-416011.internal: Container from a bad node: container_1709467795120_0004_01_000005 on host: cluster-cf9c-w-2.c.ir2024-416011.internal. Exit status: 137. Diagnostics: [2024-03-03 19:01:40.523]Container killed on request. Exit code is 137
[2024-03-03 19:01:40.526]Container exited with a non-zero exit code 137. 
[2024-03-03 19:01:40.527]Killed by external signal
.
24/03/03 19:01:40 WARN YarnSchedulerBackend$YarnSchedulerEndpoin

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.df = w2df_dict
inverted.term_total = w2tf_dict
inverted.DL = docs_dict
inverted.docs_norm = norm_dict
inverted.write_index('.', 'text_index')
index_src = "text_index.pkl"
index_dst = f'gs://{bucket_name}/postingText/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://text_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][253.4 MiB/253.4 MiB]                                                
Operation completed over 1 objects/253.4 MiB.                                    


In [ ]:
# title index
word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
docs_len = doc_title_pairs.map(lambda x: doc_len(x[0], x[1]))
docs_dict = docs_len.collectAsMap()
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>0)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
w2tf = calculate_tf(postings_filtered)
w2tf_dict = w2tf.collectAsMap()
docs_norm = doc_title_pairs.map(lambda x: doc_norm(x[0], x[1], docs_dict))
norm_dict = docs_norm.collectAsMap()


folder_name = "postings_title"
_ = partition_postings_and_write(postings_filtered, bucket_name, folder_name).collect()
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_title'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)



In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.df = w2df_dict
inverted.term_total = w2tf_dict
inverted.DL = docs_dict
inverted.write_index('.', 'title_index')
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/postings_title/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
\ [1 files][144.0 MiB/144.0 MiB]                                                
Operation completed over 1 objects/144.0 MiB.                                    


In [ ]:
#anchor_index
anchor_docs = parquetFile.select("id", "anchor_text").rdd.map(lambda r: (r[0], r[1]))
anchor = anchor_docs.flatMap(lambda x: x[1])
anchor1 = anchor.distinct().groupByKey().mapValues(lambda x: " ".join(x))
anchor2 = anchor1.flatMap(lambda x: word_count(x[1], x[0]))
postings = anchor2.groupByKey().mapValues(reduce_word_counts)
folder_name = "postings_anchor"
_ = partition_postings_and_write(postings, bucket_name, folder_name).collect()
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_anchor'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)


inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.write_index('.', 'anchor_idx')
index_src = "anchor_idx.pkl"
index_dst = f'gs://{bucket_name}/postings_anchor/{index_src}'
!gsutil cp $index_src $index_dst

24/03/03 19:51:03 WARN BlockManagerMasterEndpoint: No more replicas available for broadcast_22_python !
24/03/03 19:51:04 WARN YarnAllocator: Container from a bad node: container_1709467795120_0004_01_000012 on host: cluster-cf9c-w-2.c.ir2024-416011.internal. Exit status: 143. Diagnostics: [2024-03-03 19:51:04.216]Container killed on request. Exit code is 143
[2024-03-03 19:51:04.216]Container exited with a non-zero exit code 143. 
[2024-03-03 19:51:04.217]Killed by external signal
.
24/03/03 19:51:04 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 11 for reason Container from a bad node: container_1709467795120_0004_01_000012 on host: cluster-cf9c-w-2.c.ir2024-416011.internal. Exit status: 143. Diagnostics: [2024-03-03 19:51:04.216]Container killed on request. Exit code is 143
[2024-03-03 19:51:04.216]Container exited with a non-zero exit code 143. 
[2024-03-03 19:51:04.217]Killed by external signal
.
24/03/03 19:51:04 ERROR YarnScheduler: Lost ex

In [ ]:
#title mapping
id2title = parquetFile.select("id", "title").toPandas()
id2title.index = id2title.id
id2title = id2title['title']

file_path = 'id2title.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(id2title, file)

bucket_name = '208593988'
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.upload_from_filename(file_path)

print(f"File {file_path} uploaded to {bucket_name}.")

File id2title.pkl uploaded to 208593988.
